In [2]:
# Importar librerias
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
load_dotenv()
import time
import pandas as pd


In [3]:
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=os.getenv("SPOTIFY_CLIENT_ID"),
                                       client_secret=os.getenv("SPOTIFY_CLIENT_SECRET"))
sp = spotipy.Spotify(auth_manager=auth_manager)

In [4]:
# Prueba extraccion datos
datos_prueba = sp.search(q="genre:latin year:2014", type='artist', limit=1, offset=0)
artista_prueba = datos_prueba['artists']['items'][0]
print("Conexión exitosa con Spotify")
print("Artista de prueba encontrado:", artista_prueba['name'])

Conexión exitosa con Spotify
Artista de prueba encontrado: Anuel AA


In [8]:
# ────────────────────────────────────────────────
# 4. BÚSQUEDA DE ARTISTAS Y ÁLBUMES POR GÉNERO Y AÑO
# ────────────────────────────────────────────────

# Queremos canciones por género y año.
# Spotify no permite usar "genre:" y "year:" juntos en una sola búsqueda de canciones,
# así que dividimos el trabajo en pasos.

generos = ["flamenco", "latin", "jazz", "rock"]
año = 2014
todas_las_canciones = []
albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).

print("Buscando canciones del año", año)
print()

for genero in generos:
    print("Género:", genero)

    # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
    #    No funciona con álbumes ni canciones directamente.
    resultado_artistas = sp.search(q="genre:" + genero, type="artist",limit=50)
    artistas = resultado_artistas["artists"]["items"]

    for artista in artistas:
        nombre_artista = artista["name"]

        # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
        #    al buscar álbumes (type="album") o canciones (type="track").
        #    Aquí buscamos álbumes del artista en el año indicado.
        busqueda = "artist:" + nombre_artista + " year:" + str(año)
        resultado_albumes = sp.search(q=busqueda, type="album", limit=50)
        albumes = resultado_albumes["albums"]["items"]

        for album in albumes:
            id_album = album["id"]
            nombre_album = album["name"]
            
            # Si ya vimos este álbum, saltamos al siguiente
            if id_album in albumes_ya_vistos:
                continue
            
            # Si no lo habíamos visto, lo marcamos como visto
            albumes_ya_vistos.add(id_album)

            # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
            resultado_canciones = sp.album_tracks(id_album)
            canciones = resultado_canciones["items"]

            for cancion in canciones:
                info = {
                    "nombre": cancion["name"],
                    "artista": nombre_artista,
                    "album": nombre_album,
                    "genero": genero,
                    "año": año
                }
                todas_las_canciones.append(info)

    print("  → Canciones de este género añadidas")
    print()

# Mostramos un resumen de lo encontrado
print("RESUMEN:")

# Contamos las canciones de cada género una por una
for genero in generos:
    contador = 0
    for cancion in todas_las_canciones:
        if cancion["genero"] == genero:
            contador = contador + 1
    print(f"- {genero}: {contador} canciones")

print("Total de canciones encontradas:", len(todas_las_canciones))

Buscando canciones del año 2014

Género: flamenco
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- flamenco: 6 canciones
- latin: 2821 canciones
- jazz: 5727 canciones
- rock: 3770 canciones
Total de canciones encontradas: 12324


In [10]:
df_canciones_2014=pd.DataFrame(todas_las_canciones)
df_canciones_2014

,nombre,artista,album,genero,año
0,Life,LaRumbé,Sonido Obscuro,flamenco,2014
1,Boomerang,LaRumbé,Sonido Obscuro,flamenco,2014
2,Getaway Car,LaRumbé,Sonido Obscuro,flamenco,2014
3,Falling,LaRumbé,Sonido Obscuro,flamenco,2014
4,Dullness,LaRumbé,Sonido Obscuro,flamenco,2014
...,...,...,...,...,...
12319,Bull In A China Shop,Maná,The Moment,rock,2014
12320,Startup Kid,Maná,The Moment,rock,2014
12321,My Way,Maná,The Moment,rock,2014
12322,Thrill Of It All,Maná,The Moment,rock,2014


In [12]:
df_canciones_2014= df_canciones_2014.to_csv("canciones_2014.csv", index=False)